# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

Crew AI Documentation

In [ ]:
!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain langchain-cohere


In [ ]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [ ]:
from crewai import Agent, Task, Crew

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:777: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [ ]:
from langchain_community.chat_models import ChatCohere
import os

os.environ["COHERE_API_KEY"] = "hUQNZ8BBOMfdSMOtD3Go9QvIsvhLfxI2V9GvOcu0"
llm = ChatCohere(cohere_api_key=os.getenv("COHERE_API_KEY"))


**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = "24745cf9da56dbfef7ccb53c329c1edbf01269fcf078419e11dacd86424170f3"

## Creating Agents

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tools

### crewAI Tools

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

Simply put, DirectoryReadTool operates on a broader scale and focuses on a collection of files, whereas FileReadTool works specifically on a single file.








In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [ ]:
# زمانی استفاده می‌شود که شما بخواهید یک ابزار سفارشی ایجاد کنید
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

این ابزار به عنوان یک چک‌کننده لحن عمل می‌کند تا اطمینان حاصل شود که ایمیل‌ها و پیام‌های ایجنت دوم موثر، مثبت و سازگار با اهداف تجاری هستند.








In [ ]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"
sentiment_analysis_tool = SentimentAnalysisTool()

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2
	  # memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching for information about DeepLearningAI to gather the required data for the report.
Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company"}
Observation: ```
{
    "title": "DeepLearningAI | Artificial Intelligence Courses & Certifications",
    "text": "DeepLearningAI | Artificial Intelligence Courses & Certifications. DeepLearningAI was foun

- Display the final result as Markdown.

In [22]:
from IPython.display import Markdown
Markdown(result)

## Email 1:

Subject: Enhancing AI Education with CrewAI's Personalized Learning Solutions

Dear Andrew Ng,

I hope this email finds you well as you celebrate the successful launch of your online courses and certifications in AI and Deep Learning. At CrewAI, we are passionate about revolutionizing education, and we believe our cutting-edge technologies can significantly contribute to DeepLearningAI's mission.

As a fellow pioneer in the AI space, we understand the importance of delivering high-quality, engaging content to students worldwide. Our platform excels in creating personalized learning experiences, ensuring that every learner's journey is tailored to their unique needs and goals.

Imagine a collaboration where together, we could:
- **Co-create Compelling Courses**: By combining your industry expertise with our content development skills, we can craft interactive AI courses that captivate and educate.
- **Elevate Student Engagement**: Our platform's interactive features, such as peer collaboration and personalized learning paths, can transform the learning experience, fostering a community of engaged and motivated students.
- **Empower Instructors**: We provide comprehensive tools for instructor support, enabling efficient course management and student assessment, ultimately enhancing the overall teaching experience.
- **Scale with Confidence**: With our scalable infrastructure, DeepLearningAI can seamlessly accommodate growing student numbers, ensuring a smooth and reliable learning environment.

We believe that by integrating our solutions, DeepLearningAI can focus on what it does best—creating exceptional educational content—while we provide the technical backbone to support your vision.

Let's schedule a call to explore these possibilities further. I look forward to discussing how CrewAI can contribute to DeepLearningAI's continued success.

Best regards,
[Your Name]
Lead Sales Representative, CrewAI

## Email 2:

Subject: Unlocking AI's Potential: A Partnership for Growth

Dear Andrew,

Congratulations on the recent achievements of DeepLearningAI! As you continue to shape the future of AI education, I wanted to reach out and highlight how CrewAI's innovative technologies can further enhance your offerings.

In the ever-evolving world of AI, staying ahead requires a blend of cutting-edge technology and engaging content. Here's how we envision our partnership:
- **Content Collaboration**: Together, we can create AI courses that are not just informative but truly immersive, leveraging our interactive content development expertise.
- **White-Label Platform Integration**: Our platform can be seamlessly integrated as a white-label solution, allowing DeepLearningAI to maintain its brand identity while benefiting from our robust technical foundation.
- **Instructor Excellence**: We propose tailored training programs to empower your instructors, ensuring they stay at the forefront of AI education.
- **Community Engagement**: Through joint webinars, we can showcase the latest AI advancements, attracting and engaging a wider audience, and fostering a community of lifelong learners.

By joining forces, we can create a synergy that not only elevates the learning experience but also establishes DeepLearningAI as a leader in AI education.

I'd love to schedule a meeting to delve deeper into these ideas and tailor our solutions to DeepLearningAI's unique requirements.

Warm regards,
[Your Name]
Sales Lead, CrewAI

## Email 3:

Subject: The Future of AI Education: A CrewAI Collaboration

Hello Andrew,

As DeepLearningAI continues to make waves in the AI education sector, I wanted to share how CrewAI can play a pivotal role in taking your platform to the next level. Our solutions are designed to address the evolving needs of online learning, and I believe we can create a powerful partnership.

Here's how CrewAI can contribute to DeepLearningAI's success:
- **Interactive Learning Experiences**: Our platform is engineered to make learning AI both fun and effective. Through interactive exercises and personalized learning paths, we can enhance student motivation and retention.
- **Instructor Empowerment**: We provide a suite of tools to streamline course management, student assessment, and feedback processes, enabling your instructors to excel in their roles.
- **Scalability and Performance**: With our scalable infrastructure, DeepLearningAI can effortlessly scale to meet the demands of a growing student base, ensuring a seamless learning journey.
- **Customized Solutions**: We understand that every educational platform is unique. That's why we offer tailored solutions, ensuring our partnership aligns perfectly with DeepLearningAI's vision and values.

I propose an initial consultation to discuss these opportunities and explore how we can customize our offerings to fit DeepLearningAI's specific needs.

I look forward to the possibility of working together.

Sincerely,
[Your Name]
CrewAI Sales Team